In [1]:
# Getting some info out of WRDS
# Setting up connection, getting DJIA data

import numpy as nm
import pandas as pd
import csv
import wrds

In [2]:
#f = open('2010_2020_daily_CRSP.csv')
#csv_f = csv.reader(f)

In [4]:
#Connection to WRDS
db = wrds.Connection()


In [6]:
# WRDS test data
db.get_table('djones', 'djdaily', columns=['date', 'dji'], obs=10)

In [7]:
data = db.get_row_count('djones', 'djdaily')
data

In [43]:
db.list_tables(library='crsp')
db.describe_table(library="crsp", table="stocknames")
db.describe_table(library="crsp", table='saz_ds_dly')

Approximately 98216170 rows in crsp.saz_ds_dly.


,name,nullable,type
0,kypermno,True,DOUBLE_PRECISION
1,caldt,True,DATE
2,bidlo,True,DOUBLE_PRECISION
3,askhi,True,DOUBLE_PRECISION
4,bid,True,DOUBLE_PRECISION
5,ask,True,DOUBLE_PRECISION
6,numtrd,True,DOUBLE_PRECISION
7,openprc,True,DOUBLE_PRECISION


In [29]:
help(db.get_table)

Help on method get_table in module wrds.sql:

get_table(library, table, obs=-1, offset=0, columns=None, coerce_float=None, index_col=None, date_cols=None) method of wrds.sql.Connection instance
    Creates a data frame from an entire table in the database.
    
    :param sql: SQL code in string object.
    :param library: Postgres schema name.
    
    :param obs: (optional) int, default: -1
        Specifies the number of observations to pull from the table.
        An integer less than 0 will return the entire table.
    :param offset: (optional) int, default: 0
        Specifies the starting point for the query.
        An offset of 0 will start selecting from the beginning.
    :param columns: (optional) list or tuple, default: None
        Specifies the columns to be included in the output data frame.
    :param coerce_float: (optional) boolean, default: True
        Attempt to convert values to non-string, non-numeric objects
        to floating point. Can result in loss of prec

In [10]:
#stocknames = db.get_table(library='crsp', table='stocknames', obs=5)
#stocknames.head()

In [9]:
#stocknames = db.get_table(library='crsp', table="stocknames", columns=['permno','permco','ticker','comnam'], obs=10)
#stocknames.head()

In [11]:
#stocknames = db.get_table(library="crsp", table='saz_ds_dly', columns=['KYPERMNO','CALDT','BIDLO','ASKHI','BID','ASK'], obs=10)


In [12]:
#stocknames.head()

In [13]:
#stocknames = db.get_table(library="crsp", table='saz_ds_dly', obs=10)
#stocknames.head()

In [131]:
dly1 = db.get_table(library="crsp", table='saz_ds_dly', obs=1000)
dly2 = db.get_table(library="crsp", table='saz_dp_dly',obs=1000)
dly3 = pd.merge(dly1, dly2, on=["kypermno", "caldt"])
dly3.head()

,kypermno,caldt,bidlo,askhi,bid,ask,numtrd,openprc,prc,ret,retx,tcap,vol
0,10000.0,1986-01-07,2.375,2.750,NaN,NaN,NaN,None,-2.5625,NaN,NaN,9430.0,1000.0
1,10000.0,1986-01-08,2.375,2.625,NaN,NaN,NaN,None,-2.5000,-0.02439,-0.02439,9200.0,12800.0
2,10000.0,1986-01-09,2.375,2.625,NaN,NaN,NaN,None,-2.5000,0.00000,0.00000,9200.0,1400.0
3,10000.0,1986-01-10,2.375,2.625,NaN,NaN,NaN,None,-2.5000,0.00000,0.00000,9200.0,8500.0
4,10000.0,1986-01-13,2.500,2.750,NaN,NaN,NaN,None,-2.6250,0.05000,0.05000,9660.0,5450.0


In [132]:
dly3.drop(columns=['prc','ret','retx','tcap'])

,kypermno,caldt,bidlo,askhi,bid,ask,numtrd,openprc,vol
0,10000.0,1986-01-07,2.375,2.750,NaN,NaN,NaN,None,1000.0
1,10000.0,1986-01-08,2.375,2.625,NaN,NaN,NaN,None,12800.0
2,10000.0,1986-01-09,2.375,2.625,NaN,NaN,NaN,None,1400.0
3,10000.0,1986-01-10,2.375,2.625,NaN,NaN,NaN,None,8500.0
4,10000.0,1986-01-13,2.500,2.750,NaN,NaN,NaN,None,5450.0
...,...,...,...,...,...,...,...,...,...
995,10001.0,1988-07-12,6.250,6.250,6.25,6.75,1.0,None,300.0
996,10001.0,1988-07-13,6.250,6.750,6.25,6.75,0.0,None,0.0
997,10001.0,1988-07-14,6.250,6.750,6.25,6.75,0.0,None,0.0
998,10001.0,1988-07-15,6.250,6.750,6.25,6.75,0.0,None,0.0


In [133]:
for row in dly3.itertuples():
    permno = row.kypermno*1
    dly3.to_csv(str(permno)+'.csv', index = False)

In [150]:
df['kypermno'].unique()

array([10000., 10001.])

In [147]:
df=dly3

In [151]:
for permco in df['kypermno'].unique():
    sub_df = df[df['kypermno'] == permco]
    sub_df.to_csv(f'{permco}.csv', index=False)

In [152]:
df['prc'].plot()